In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker==2.117.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
yapf 0.40.1 requires importlib-metadata>=6.6.0, but you have importlib-metadata 4.13.0 which is incompatible.


In [3]:
import sagemaker
import pandas as pd
#from sklearn.datasets import load_boston
from sagemaker.sklearn.estimator import SKLearn
from sklearn.model_selection import train_test_split

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()


print(f"(role:{role})")
print(f"(bucket:{bucket})")
print(f"(sess:{sess})")

(role:arn:aws:iam::913060180538:role/service-role/AmazonSageMaker-ExecutionRole-20240215T143909)
(bucket:sagemaker-us-east-1-913060180538)
(sess:<sagemaker.session.Session object at 0x7fd9a31de2c0>)


In [29]:
role

'arn:aws:iam::913060180538:role/service-role/AmazonSageMaker-ExecutionRole-20240215T143909'

In [ ]:
tracking_uri = 'http://3.84.145.209:5000' 

# Prepare dataset

In [31]:
import os
os.listdir()

['boston_housing_sagemaker_mlflow.ipynb',
 '.ipynb_checkpoints',
 'source_dir',
 '.sparkmagic',
 'test.csv',
 'bank_clean.csv',
 'HousingData.csv',
 'train.csv',
 'BANK_APPLICATION_AWS_SAGEMAKER.ipynb',
 'lost+found']

In [32]:
df = pd.read_csv("HousingData.csv")

In [33]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [47]:
features = df.columns.tolist()[:-1]

In [48]:
features

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [39]:
X = df.drop("MEDV",axis = 1)
y = df["MEDV"]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [49]:
X_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
182,0.09103,0.0,2.46,0.0,0.4880,7.155,92.2,2.7006,3,193,17.8,394.12,4.82
155,3.53501,0.0,19.58,1.0,0.8710,6.152,NaN,1.7455,5,403,14.7,88.01,15.02
280,0.03578,20.0,3.33,0.0,0.4429,7.820,64.5,4.6947,5,216,14.9,387.31,3.76
126,0.38735,0.0,25.65,0.0,0.5810,5.613,NaN,1.7572,2,188,19.1,359.29,27.26
329,0.06724,0.0,3.24,0.0,0.4600,6.333,17.2,5.2146,4,430,16.9,375.21,7.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.17120,0.0,8.56,0.0,0.5200,5.836,91.9,2.2110,5,384,20.9,395.67,18.66
270,0.29916,20.0,6.96,0.0,0.4640,5.856,42.1,4.4290,3,223,18.6,388.65,13.00
348,0.01501,80.0,2.01,0.0,0.4350,6.635,29.7,8.3440,4,280,17.0,390.94,5.99
435,11.16040,0.0,18.10,0.0,0.7400,6.629,94.6,2.1247,24,666,20.2,109.85,23.27


In [50]:
y_train

182    37.9
155    15.6
280    45.4
126    15.7
329    22.6
       ... 
106    19.5
270    21.1
348    24.5
435    13.4
102    18.6
Name: MEDV, Length: 379, dtype: float64

In [74]:
trainX = pd.DataFrame(X_train, columns=features)
trainX['target'] = y_train

testX = pd.DataFrame(X_test, columns=features)
testX['target'] = y_test

trainX.reset_index(drop=True, inplace=True)
testX.reset_index(drop=True, inplace=True)

# Assuming trainX and testX are your DataFrames
trainX.dropna(inplace=True)
testX.dropna(inplace=True)


trainX.to_csv('boston_train.csv', index=False)
testX.to_csv('boston_test.csv', index=False)


In [75]:
df_train = pd.read_csv('boston_train.csv')

In [76]:
df_train.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

In [77]:

df_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.09103,0.0,2.46,0.0,0.4880,7.155,92.2,2.7006,3,193,17.8,394.12,4.82,37.9
1,0.03578,20.0,3.33,0.0,0.4429,7.820,64.5,4.6947,5,216,14.9,387.31,3.76,45.4
2,0.06724,0.0,3.24,0.0,0.4600,6.333,17.2,5.2146,4,430,16.9,375.21,7.34,22.6
3,1.35472,0.0,8.14,0.0,0.5380,6.072,100.0,4.1750,4,307,21.0,376.73,13.04,14.5
4,0.22212,0.0,10.01,0.0,0.5470,6.092,95.4,2.5480,6,432,17.8,396.90,17.09,18.7


In [78]:
df_train.shape

(288, 14)

In [79]:
df_test = pd.read_csv('boston_test.csv')

In [80]:
df_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4,254,17.6,396.90,3.53,32.4
1,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4,711,20.1,390.11,18.07,13.6
2,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4,305,19.2,390.91,5.52,22.8
3,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24,666,20.2,385.09,17.27,16.1
4,0.10153,0.0,12.83,0.0,0.437,6.279,74.5,4.0522,5,398,18.7,373.66,11.97,20.0


In [81]:
df_test.shape

(106, 14)

In [82]:
df_test.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

# Upload the dataset to S3 bucket

In [83]:
# send data to S3. SageMaker will take training data from s3
train_path = sess.upload_data(path='boston_train.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')
test_path = sess.upload_data(path='boston_test.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')

In [84]:
train_path

's3://sagemaker-us-east-1-913060180538/sagemaker/sklearncontainer/boston_train.csv'

In [85]:
test_path

's3://sagemaker-us-east-1-913060180538/sagemaker/sklearncontainer/boston_test.csv'

# Training the model

In [86]:
hyperparameters = {
    'tracking_uri': tracking_uri,
    'experiment_name': 'boston-housing',
    'n-estimators': 100,
    'min-samples-leaf': 3,
    'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
    'target': 'target'
}

metric_definitions = [{'Name': 'median-AE', 'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}]

estimator = SKLearn(
    entry_point='train.py',
    source_dir='source_dir',
    role=role,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.0-1',
    base_job_name='mlflow',
)

In [104]:
estimator.fit({'train':train_path, 'test': test_path})

2024-02-23 22:35:23 Starting - Starting the training job...
2024-02-23 22:35:46 Starting - Preparing the instances for trainingProfilerReport-1708727723: InProgress
...
2024-02-23 22:36:12 Downloading - Downloading input data...
2024-02-23 22:36:46 Downloading - Downloading the training image...
2024-02-23 22:37:12 Training - Training image download completed. Training in progress..2024-02-23 22:37:19,646 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-02-23 22:37:19,648 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-02-23 22:37:19,651 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-02-23 22:37:19,666 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-02-23 22:37:19,902 sagemaker-training-toolkit INFO     Installing module with the following command:
/miniconda3/bin/python -m pip install . -r requirements.txt
Processing /opt/ml

In [89]:
!pip install -q mlflow==2.6.0

In [91]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
#from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

In [92]:
region_name = 'us-east-1'

In [98]:
#image_uri = "913060180538.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"

In [97]:
image_uri = "913060180538.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc"

In [95]:
!mlflow sagemaker build-and-push-container

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2024/02/23 22:03:46 INFO mlflow.models.docker_utils: Building docker image with name mlflow-pyfunc
Sending build context to Docker daemon  4.096kB

Step 1/26 : FROM ubuntu:20.04
20.04: Pulling from library/ubuntu
8ee087424735: Pulling fs layer
8ee087424735: Verifying Checksum
8ee087424735: Download complete
8ee087424735: Pull complete
Digest: sha256:bb1c41682308d7040f74d103022816d41c50d7b0c89e9d706a74b4e548636e54
Status: Downloaded newer image for ubuntu:20.04
 ---> 18ca3f4297e7
Step 2/26 : RUN apt-get -y update
 ---> Running in c6b742789be6
Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubun

In [101]:
endpoint_name = 'boston-housing'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 's3://sagemaker-us-east-1-913060180538/mlflow-2024-02-23-21-36-35-410/source/sourcedir.tar.gz'

In [102]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region_name
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

MlflowException: Failed to find MLmodel configuration within the specified model's root directory.